In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from datetime import datetime
from IPython.core.display import display, HTML
import cv2
from PIL import Image
from pathlib import Path
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import ast

from sklearn.model_selection import train_test_split

from utils import *
from model_training_utils import train as training_loop
from model_training_utils import validate as validation_loop
from model_training_utils import hp_grid_search
from models import CNN

In [2]:
t= torch.tensor(cv2.imread('./images/resized/1.jpg'))

In [3]:
t_movedim= t.movedim(2, 0)

In [4]:
t_reshape= t.reshape((3,500,500))

In [5]:
df= pd.read_csv('./datasets/annotations_map.csv', converters={'new_bb': from_np_array})
df_train = df.reset_index()
X = df_train[['new_path','new_bb']]
Y = df_train['class']
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.25, random_state=42)
print('Number of training observations: ', X_train.shape[0])
print('Number of validation observations: ', X_val.shape[0])

Number of training observations:  32
Number of validation observations:  11


In [7]:
## Define Hyperparameters -- Currently setting values that we can modify
loss_type = "l1"
learning_rate = 0.001
momentum = 0.5
reg = 1e-2
training_batch_size= 12
validation_batch_size= 5
model= "SimpleCNN"

criterion = nn.L1Loss()
model= CNN()
train_ds = WaldoDataset(X_train['new_path'],X_train['new_bb'] ,y_train)
valid_ds = WaldoDataset(X_val['new_path'],X_val['new_bb'],y_val)
train_dl = DataLoader(train_ds, batch_size=training_batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=validation_batch_size)
optimizer = torch.optim.SGD(model.parameters(), learning_rate,momentum=momentum,weight_decay=reg)

In [8]:
training_loop(model, optimizer, train_dl, valid_dl, criterion, 2, False, False, True)

/Users/andrew.lofgreen/Documents/OMSCS/cs7643/dl7643/CS7643-Final-Project/model_training_utils.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_bb= torch.tensor(y_bb)


 
--------------------------------------------------------
Training Loss for Epoch 0: 83.59792900085449


/Users/andrew.lofgreen/Documents/OMSCS/cs7643/dl7643/CS7643-Final-Project/model_training_utils.py:75: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_bb= torch.tensor(y_bb)


Validation Loss for Epoch 0: 614.0933227539062
 
--------------------------------------------------------
Training Loss for Epoch 1: 81.57487869262695
Validation Loss for Epoch 1: 609.1585235595703


In [5]:
img= cv2.imread("./images/resized/1.jpg")
img= torch.tensor(img)
img= torch.unsqueeze(img, dim= 0)
img= img.float()
df= pd.read_csv('./datasets/annotations_map.csv')
df.head(10)

,filename,width,height,class,xmin,ymin,xmax,ymax,new_path,new_bb
0,./images/base/1.jpg,2048,1251,waldo,706,513,743,562,images/resized/1.jpg,[172 205 181 225]
1,./images/base/10.jpg,1600,980,waldo,715,157,733,181,images/resized/10.jpg,[223 80 229 92]
2,./images/base/11.jpg,2828,1828,waldo,460,1530,482,1557,images/resized/11.jpg,[ 81 418 85 426]
3,./images/base/12.jpg,1276,1754,waldo,846,517,878,563,images/resized/12.jpg,[332 147 344 160]
4,./images/base/13.jpg,1280,864,waldo,932,274,942,288,images/resized/13.jpg,[364 159 368 167]
5,./images/base/14.jpg,1700,2340,waldo,197,1878,237,1919,images/resized/14.jpg,[ 58 401 70 410]
6,./images/base/15.jpg,1600,1006,waldo,297,428,311,445,images/resized/15.jpg,[ 93 213 97 221]
7,./images/base/17.jpg,1599,1230,waldo,751,250,772,281,images/resized/17.jpg,[235 102 241 114]
8,./images/base/18.jpg,1590,981,waldo,1259,75,1290,111,images/resized/18.jpg,[396 38 406 57]
9,./images/base/19.jpg,1280,864,waldo,338,623,351,640,images/resized/19.jpg,[132 361 137 370]


In [6]:
output= model(img)

In [7]:
output

tensor([[13458.9287,  9587.8594, 15120.9600, 10811.1416]],
       grad_fn=<AddmmBackward>)

In [58]:
criterion(output, torch.tensor([[172,205,181,225]]))

tensor(15391.2998, grad_fn=<L1LossBackward>)